### Nested Sampling

Let's try Nested sampling method using dynesty library for posterior sampling and model testing. \
The models are:
1. $f_1(t) = b + A e^{-\alpha (t-t_0)}$
2. $f_2(t) = b + A e^{-\frac{(t-t_0)^2}{2\sigma^2}}$

In [1]:
import numpy as np

In [2]:
data = np.load(r"C:\Users\ricca\Documents\Unimib-Code\AstroStatistics\AML\Notebooks\AstroStatistics\Data\transient.npy")

In [3]:
Time = data.T[0]
vals = data.T[1]
errs = data.T[2]

In [4]:
def burst(theta,t):
    A,b,t0,alpha=theta 
    return np.where(t<t0,b,b+A*np.exp(-alpha*(t-t0)))

def gprofile(theta,t):
    A,b,t0,sigmaW=theta     
    return b+A*np.exp(-(((t-t0)/sigmaW)**2 ) / 2)

In [5]:
t0min,t0max = 0,100
Amin,Amax=0,50
bmin,bmax=0,50
alphamin,alphamax=np.exp(-5),np.exp(5)
sigmaWmin,sigmaWmax=np.exp(-2),np.exp(2)

In [11]:
import scipy

ndim = 4

def loglike(theta, data, model):
    x, y, sigma_y = data.T
    if model =='burst':
        y_fit = burst(theta, x)
    elif model == 'gprofile':
        y_fit = gprofile(theta, x)

    # Note I'm not tracking the normalization here! 
    # Ok because here I only care about the ratio between two evidences, not the evidence itself
    return -0.5 * np.sum((y-y_fit)**2 / sigma_y**2 ) 


def ptform(u,model):
    """Transforms the uniform random variables `u ~ Unif[0., 1.)`
    to the parameters of interest."""

    x = np.array(u)  # copy u

    x[0] = scipy.stats.uniform(loc=Amin,scale=Amax-Amin).ppf(u[0])
    x[1] = scipy.stats.uniform(loc=bmin,scale=bmax-bmin).ppf(u[1])
    x[2] = scipy.stats.uniform(loc=t0min,scale=t0max-t0min).ppf(u[2])
   
    if model =='burst':
        x[3] = scipy.stats.loguniform.ppf(u[3],alphamin,alphamax)
    elif model =='gprofile':
        x[3] = scipy.stats.loguniform.ppf(u[3],sigmaWmin,sigmaWmax)
        #x[3] = scipy.stats.uniform(loc=sigmaWmin,scale=sigmaWmax-sigmaWmin).ppf(u[3])

    return x

In [12]:
import warnings
import dynesty

In [ ]:
rfig, raxes = dyplot.runplot(sresults)
plt.savefig(r'C:\Users\ricca\Documents\Unimib-Code\AstroStatistics\AML\Notebooks\AstroStatistics\ModelTest\live1.png')

In [ ]:
tfig, taxes = dyplot.traceplot(sresults, quantiles=[0.16, 0.5, 0.84], post_color='black', connect=True,  use_math_text=True, show_titles=True, title_kwargs={"fontsize": 12}, label_kwargs={"fontsize": 14}, truth_color='red', kde=False, trace_cmap='inferno')

tfig.suptitle("Dynesty Run Summary", fontsize=16)
taxes[0, 0].set_ylabel("A", fontsize=12)
taxes[1, 0].set_ylabel("b", fontsize=12)
taxes[2, 0].set_ylabel(r"$\alpha$", fontsize=12)
taxes[3, 0].set_ylabel(r"$t_0$", fontsize=12)
taxes[0, 1].set_xlabel("A", fontsize=12)
taxes[1, 1].set_xlabel("b", fontsize=12)
taxes[2, 1].set_xlabel(r"$\alpha$", fontsize=12)
taxes[3, 1].set_xlabel(r"$t_0$", fontsize=12)
tfig.set_size_inches(15, 10)
tfig.tight_layout()

plt.savefig(r'C:\Users\ricca\Documents\Unimib-Code\AstroStatistics\AML\Notebooks\AstroStatistics\ModelTest\traceplot1.png')

In [ ]:
cfig, caxes = dyplot.cornerplot(sresults, color='black', quantiles=[0.16, 0.5, 0.84], show_titles=True)

caxes[0, 0].set_ylabel("A", fontsize=12)
caxes[1, 0].set_ylabel("b", fontsize=12)
caxes[2, 0].set_ylabel(r"$\alpha$", fontsize=12)
caxes[3, 0].set_ylabel(r"$t_0$", fontsize=12)

caxes[3, 0].set_xlabel("A", fontsize=12)
caxes[3, 1].set_xlabel("b", fontsize=12)
caxes[3, 2].set_xlabel(r"$\alpha$", fontsize=12)
caxes[3, 3].set_xlabel(r"$t_0$", fontsize=12)
cfig.suptitle("Dynesty Corner Plot", fontsize=16)
cfig.set_size_inches(15, 10)
cfig.tight_layout()

plt.savefig(r'C:\Users\ricca\Documents\Unimib-Code\AstroStatistics\AML\Notebooks\AstroStatistics\ModelTest\corner1.png')

In [ ]:
samples = sresults.samples  # samples
weights = np.exp(sresults.logwt - sresults.logz[-1])  # normalized weights

labels = ["A","b","t0","alpha"]

samples_equal = dyfunc.resample_equal(samples, weights)
corner.corner(samples_equal,labels=labels);

In [ ]:
quantiles = [dyfunc.quantile(samps, [0.05, 0.5, 0.95], weights=weights)
             for samps in samples.T]
for q,l in zip(quantiles,labels):
    low,med,up=q
    print(l+"   "+str(med)+" +"+str(up-med)+" -"+str(med-low))

In [ ]:
sresults.summary()

In [14]:
with warnings.catch_warnings():
    # Potentially dangerous, but hey I know what I'm doing. 
    # Those warnings come from regions where the likelihood is ~zero
    warnings.simplefilter("ignore", category=RuntimeWarning)

    # Define and run sampler
    sampler = dynesty.NestedSampler(loglike, ptform, ndim,logl_args=[data,'burst'],ptform_args=['burst'],nlive=1000,bound='balls')
    sampler.run_nested()
    sresults = sampler.results

7309it [11:55, 10.22it/s, bound: 24 | nc: 5 | ncall: 71239 | eff(%): 10.260 | loglstar:   -inf < -87.105 <    inf | logz: -93.520 +/-  0.076 | dlogz: 37.212 >  1.009]       


KeyboardInterrupt: 